In [ ]:
!python.exe -m pip install --upgrade pip

In [ ]:
!pip install requests openpyxl

In [ ]:
import os
import requests
import openpyxl

def download_images_from_file(file_path, output_folder, failed_images_wb):
    # Tạo thư mục đầu ra nếu chưa tồn tại
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with open(file_path, 'r') as file:
        # Lấy tên của file txt (bỏ phần đuôi .txt)
        file_name = os.path.splitext(os.path.basename(file_path))[0]

        # Tạo thư mục con trong thư mục đầu ra với tên của file txt
        output_subfolder = os.path.join(output_folder, file_name)
        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)

        lines = file.readlines()
        for line in lines:
            # Tách tên tệp và đường dẫn ảnh từ mỗi dòng trong file txt
            image_name, image_url = line.strip().split('\t')

            # Tạo đường dẫn lưu trữ hình ảnh đã tải xuống
            save_path = os.path.join(output_subfolder, image_name.split('/')[1])

            # Tải xuống hình ảnh và lưu vào thư mục con tương ứng
            try:
                response = requests.get(image_url)
                response.raise_for_status()
                with open(save_path, 'wb') as img_file:
                    img_file.write(response.content)
                print(f"Tải xuống {image_name} thành công.")
            except requests.exceptions.RequestException as e:
                print(f"Lỗi trong quá trình tải xuống {image_name}: {e}")
                # Lưu thông tin file không tải được vào file Excel
                failed_images_wb.append([file_name, image_name, image_url, str(e)])

# Đường dẫn đến thư mục chứa các file txt
folder_path = 'yoga_dataset_links'

# Lấy danh sách các file txt trong thư mục
file_list = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Thư mục đầu ra để lưu trữ hình ảnh
output_folder = 'output_images'

# Khởi tạo một workbook để lưu thông tin về các file không tải được
failed_images_wb = openpyxl.Workbook()
failed_images_sheet = failed_images_wb.active
failed_images_sheet.append(['File txt', 'Tên ảnh', 'Đường dẫn', 'Lỗi'])

# Duyệt qua từng file txt và thực hiện tải xuống
for file in file_list:
    file_path = os.path.join(folder_path, file)
    download_images_from_file(file_path, output_folder, failed_images_sheet)

# Lưu workbook chứa thông tin về các file không tải được vào file Excel
failed_images_wb.save('failed_images.xlsx')


# Split file 

In [ ]:
import os
import shutil
import math
import re

def split_files_into_folders(input_folder, output_folder_base, files_per_folder):
    # Tạo thư mục đầu ra nếu chúng không tồn tại
    if not os.path.exists(output_folder_base):
        os.makedirs(output_folder_base)

    # Lấy danh sách tất cả các tệp tin trong thư mục đầu vào và sắp xếp theo tên
    file_list = os.listdir(input_folder)
    file_list.sort(key=lambda x: int(re.sub('\D', '', x)))  # Sắp xếp theo số trong tên tệp
    total_files = len(file_list)

    # Tính toán số lượng thư mục cần thiết
    num_folders = math.ceil(total_files / files_per_folder)

    # Di chuyển tệp tin vào các thư mục đầu ra
    for i in range(num_folders):
        # Tạo thư mục con
        output_folder = os.path.join(output_folder_base, f"yoga_dataset_links {i + 1}")
        os.makedirs(output_folder, exist_ok=True)

        # Lấy danh sách các tệp tin cần di chuyển vào thư mục đầu ra
        start_idx = i * files_per_folder
        end_idx = min((i + 1) * files_per_folder, total_files)
        files_to_move = file_list[start_idx:end_idx]

        # Di chuyển các tệp tin vào thư mục đầu ra
        for file_name in files_to_move:
            src_path = os.path.join(input_folder, file_name)
            dst_path = os.path.join(output_folder, file_name)
            shutil.copy(src_path, dst_path)

In [ ]:
# Sử dụng hàm để chia các tệp trong thư mục "yoga_dataset_links" thành các thư mục đầu ra
# với số lượng tệp tin mỗi thư mục là 4
split_files_into_folders("yoga_dataset_links", "./", 4)

# Combined xlsx

In [ ]:
import openpyxl
import os
import re

def combine_xlsx_files(folder_path, combined_xlsx_path):
    # Lấy danh sách tên các tệp Excel trong thư mục
    xlsx_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
    xlsx_files.sort(key=lambda x: int(re.sub('\D', '', x)))  # Sắp xếp theo số trong tên tệp
    
    # Tạo một Workbook mới
    combined_workbook = openpyxl.Workbook()
    combined_sheet = combined_workbook.active
    combined_sheet.append(['File txt', 'Tên ảnh', 'Đường dẫn', 'Lỗi'])

    # Duyệt qua từng tệp Excel và sao chép nội dung vào tệp kết hợp
    for xlsx_file in xlsx_files:
        xlsx_file_path = os.path.join(folder_path, xlsx_file)
        wb = openpyxl.load_workbook(xlsx_file_path)
        sheet = wb.active

        # Duyệt qua từng hàng và cột trong tệp Excel và sao chép dữ liệu
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, values_only=True):
            combined_sheet.append(row)

    # Lưu tệp kết hợp
    combined_workbook.save(combined_xlsx_path)

    print(f"Đã nối và lưu thành công vào {combined_xlsx_path}")


In [ ]:
folder_path = "failed_images"
combined_xlsx_path = "failed_images.xlsx"
combine_xlsx_files(folder_path, combined_xlsx_path)